In [178]:
import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np

In [179]:
empresas = pd.read_csv('empresas.csv')
infos = pd.read_csv('infos_empresas.csv')
cotacoes = pd.read_csv('cotacoes.csv')
dividendos = pd.read_csv('dividendos.csv')

empresas = empresas.drop(empresas.columns[0], axis=1)
infos = infos.drop(infos.columns[0], axis=1)
cotacoes = cotacoes.drop(cotacoes.columns[0], axis=1)
dividendos = dividendos.drop(dividendos.columns[0], axis=1)

In [180]:
infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='bfill')

C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\3737537357.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='bfill')


In [181]:
infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='ffill')

C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\1645137256.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='ffill')


In [182]:
anos = np.arange(2015,2023)
anos

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [183]:
empresas

,empresa,setor,codigo_cvm,estatal,acoes_on,acoes_pn,acoes_total,equivalencia_unit
0,CMIG4,energia,2453,True,735847624,1465523064,2201370688,1
1,CMIG3,energia,2453,True,735847624,1465523064,2201370688,1
2,TAEE11,energia,20257,False,590714069,442782652,1033496721,3
3,TAEE3,energia,20257,False,590714069,442782652,1033496721,1
4,TAEE4,energia,20257,False,590714069,442782652,1033496721,1
5,CPLE6,energia,14311,True,1300347300,1682463291,2982810591,1
6,CPLE3,energia,14311,True,1300347300,1682463291,2982810591,1
7,CSMG3,saneamento,19445,True,380253069,0,380253069,1
8,SAPR11,saneamento,18627,True,503735259,1007470260,1511205519,5
9,SAPR3,saneamento,18627,True,503735259,1007470260,1511205519,1


In [184]:
df_list = []
for i in range(len(empresas)):
    ticker = empresas.iloc[i]['empresa']
    cod_cvm = empresas.iloc[i]['codigo_cvm']
    n_de_acoes = empresas.iloc[i]['acoes_total']
    setor = empresas.iloc[i]['setor']
    estatal = empresas.iloc[i]['estatal']
    
    for ano in anos:
        dados_empresa = infos[infos['cvm_code']==cod_cvm]
        dados_empresa_ano = dados_empresa[dados_empresa['year']==ano]

        try:
            lucro_liquido = dados_empresa_ano[dados_empresa_ano['data']=='net_income']['value'].values[0]
        except:
            lucro_liquido = 1
        try:
            patrimonio_liquido = dados_empresa_ano[dados_empresa_ano['data']=='equity']['value'].values[0]
        except:
            patrimonio_liquido = 1
        try:
            divida_liquida = dados_empresa_ano[dados_empresa_ano['data']=='net_debt']['value'].values[0]
        except:
            divida_liquida = 1

        roe = lucro_liquido/patrimonio_liquido
        debt_over_income = divida_liquida/lucro_liquido
        vpa = patrimonio_liquido/n_de_acoes
        lpa = lucro_liquido/n_de_acoes

        dicionario = {
            'ticker' : ticker,
            'cvm_code' : cod_cvm,
            'setor' : setor,
            'estatal' : estatal,
            'year' : ano,
            'roe' : roe,
            'debt_ratio' : debt_over_income,
            'vpa' : vpa,
            'lpa' : lpa
        }

        df = pd.DataFrame(dicionario, index=[0])
        df_list.append(df)

C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\2447898130.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  roe = lucro_liquido/patrimonio_liquido
C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\2447898130.py:26: RuntimeWarning: invalid value encountered in scalar divide
  roe = lucro_liquido/patrimonio_liquido
C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\2447898130.py:27: RuntimeWarning: divide by zero encountered in scalar divide
  debt_over_income = divida_liquida/lucro_liquido


In [185]:
indicadores = pd.concat(df_list)
indicadores

,ticker,cvm_code,setor,estatal,year,roe,debt_ratio,vpa,lpa
0,CMIG4,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575
0,CMIG4,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066
0,CMIG4,2453,energia,True,2017,0.069894,12.289442,6.509642,0.454987
0,CMIG4,2453,energia,True,2018,0.109271,7.565812,7.240637,0.791195
0,CMIG4,2453,energia,True,2019,0.198376,4.226188,7.314763,1.451075
...,...,...,...,...,...,...,...,...,...
0,TIMS3,24929,telecom,False,2018,0.079137,-1.500065,8.005929,0.633563
0,TIMS3,24929,telecom,False,2019,NaN,-inf,0.000000,0.000000
0,TIMS3,24929,telecom,False,2020,0.079529,-1.247876,9.576464,0.761602
0,TIMS3,24929,telecom,False,2021,NaN,inf,0.000000,0.000000


In [186]:
dividendos_list = []
for i in range(len(indicadores)):
    ticker = indicadores.iloc[i]['ticker']
    ano = indicadores.iloc[i]['year']

    dividendos_ticker_ano = dividendos[dividendos['ticker']==ticker][dividendos[dividendos['ticker']==ticker]['year']==ano]
    dividendo_ano_valor = dividendos_ticker_ano['value'].sum()

    dividendos_list.append(dividendo_ano_valor)

In [187]:
indicadores['dividendo'] = dividendos_list

In [188]:
indicadores

,ticker,cvm_code,setor,estatal,year,roe,debt_ratio,vpa,lpa,dividendo
0,CMIG4,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575,1.219627
0,CMIG4,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066,1.293778
0,CMIG4,2453,energia,True,2017,0.069894,12.289442,6.509642,0.454987,0.487119
0,CMIG4,2453,energia,True,2018,0.109271,7.565812,7.240637,0.791195,1.288606
0,CMIG4,2453,energia,True,2019,0.198376,4.226188,7.314763,1.451075,1.450221
...,...,...,...,...,...,...,...,...,...,...
0,TIMS3,24929,telecom,False,2018,0.079137,-1.500065,8.005929,0.633563,0.236894
0,TIMS3,24929,telecom,False,2019,NaN,-inf,0.000000,0.000000,0.465888
0,TIMS3,24929,telecom,False,2020,0.079529,-1.247876,9.576464,0.761602,0.549765
0,TIMS3,24929,telecom,False,2021,NaN,inf,0.000000,0.000000,0.432831


In [189]:
c4 = indicadores[indicadores['ticker']=='TAEE11']['dividendo'].mean()
c3 = indicadores[indicadores['ticker']=='TAEE4']['dividendo'].mean()
print(c4, c3)
res = c3/c4
print(res)

8.851015488000002 2.989312929375
0.3377367188463109


In [190]:
equivalencia_empresas_ticker_list = []
equivalencia_empresas_valor_list = []
for cvm_code in empresas['codigo_cvm'].unique():
    ticker_list = empresas[empresas['codigo_cvm']==cvm_code]['empresa'].values
    ticker_principal = ticker_list[0]
    
    if len(ticker_list)==1:
        equivalencia_empresas_ticker_list.append(ticker_principal)
        equivalencia_empresas_valor_list.append(1)
    else:
        for ticker in ticker_list:
            dividendos_ticker = indicadores[indicadores['ticker']==ticker]['dividendo'].mean()
            dividendos_ticker_principal = indicadores[indicadores['ticker']==ticker_principal]['dividendo'].mean()
            multiplicador = dividendos_ticker/dividendos_ticker_principal

            equivalencia_empresas_ticker_list.append(ticker_principal)
            equivalencia_empresas_valor_list.append(multiplicador)


In [191]:
empresas['equivalencia_ticker'] = equivalencia_empresas_ticker_list
empresas['equivalencia_valor'] = equivalencia_empresas_valor_list

In [192]:
empresas

,empresa,setor,codigo_cvm,estatal,acoes_on,acoes_pn,acoes_total,equivalencia_unit,equivalencia_ticker,equivalencia_valor
0,CMIG4,energia,2453,True,735847624,1465523064,2201370688,1,CMIG4,1.000000
1,CMIG3,energia,2453,True,735847624,1465523064,2201370688,1,CMIG4,0.924145
2,TAEE11,energia,20257,False,590714069,442782652,1033496721,3,TAEE11,1.000000
3,TAEE3,energia,20257,False,590714069,442782652,1033496721,1,TAEE11,0.337737
4,TAEE4,energia,20257,False,590714069,442782652,1033496721,1,TAEE11,0.337737
5,CPLE6,energia,14311,True,1300347300,1682463291,2982810591,1,CPLE6,1.000000
6,CPLE3,energia,14311,True,1300347300,1682463291,2982810591,1,CPLE6,0.908667
7,CSMG3,saneamento,19445,True,380253069,0,380253069,1,CSMG3,1.000000
8,SAPR11,saneamento,18627,True,503735259,1007470260,1511205519,5,SAPR11,1.000000
9,SAPR3,saneamento,18627,True,503735259,1007470260,1511205519,1,SAPR11,0.290275


In [193]:
multiplos = {
    'energia': {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7},
    'saneamento' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7},
    'banco' : {'vpa':1.4,
                'roe': 0.15,
                'pl': 10,
                'dy': 8,
                'estatal_mult': 0.6},
    'seguro' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 12,
                'dy': 8,
                'estatal_mult': 0.7},
    'telecom' : {'vpa':1.0,
                'roe': 0.10,
                'pl': 10,
                'dy': 8,
                'estatal_mult': 0.7},
    'combustivel' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7}
}

In [201]:
df_list_preco_teto = []
for i in range(len(empresas)):
    ticker = empresas.iloc[i]['empresa']
    setor = empresas.iloc[i]['setor']
    estatal = empresas.iloc[i]['estatal']
    equivalencia = empresas.iloc[i]['equivalencia_unit']

    df = indicadores[indicadores['ticker']==ticker]
    lpa = df['lpa'].iloc[-1]
    vpa = df['vpa'].iloc[-1]
    div = df['dividendo'].iloc[-1]


    indice_pvp = multiplos[setor]['vpa']
    if estatal:
        indice_pvp = indice_pvp * multiplos[setor]['estatal_mult']

    indice_pl = multiplos[setor]['pl']
    if estatal:
        indice_pl = indice_pl * multiplos[setor]['estatal_mult']

    indice_div = multiplos[setor]['dy']
    if estatal:
        indice_div = indice_div / multiplos[setor]['estatal_mult']

    preco_vpa = indice_pvp * vpa
    preco_pl = indice_pl * lpa
    preco_div = indice_div * div

    print(preco_vpa*equivalencia, preco_pl*equivalencia, preco_div)
    

8.312068267168641 9.113593821051186 35.02315942857143
8.312068267168641 9.113593821051186 35.02315942857143
22.887071743307448 29.447132614560083 116.51035749600001
7.629023914435816 9.815710871520027 38.836785816
7.629023914435816 9.815710871520027 38.836785816
5.950840141696412 1.8880424110710823 20.2426928
5.950840141696412 1.8880424110710823 18.402293028571428
16.025621505240235 10.867693483362784 4.392365977142857
24.42085139049839 18.66899018385599 51.72254914285714
4.8841702780996785 3.733798036771198 9.578249828571428
4.8841702780996785 3.733798036771198 10.536074845714285
0.0 22.37598752213481 10.773714285714286
19.55392693123977 12.356399443479209 35.256884400000004
19.55392693123977 12.356399443479209 35.731038000000005
19.55392693123977 12.356399443479209 35.256884400000004
10.374462600085128 13.7042943603398 9.824616
10.374462600085128 13.7042943603398 9.824615999999999
41.32869367243219 38.24609087425915 53.800932696000004
20.664346836216094 19.123045437129576 25.61949177

In [195]:
df_list_indicadores_val = []
for i in range(len(indicadores)):
    ticker = indicadores.iloc[i]['ticker']
    ano = indicadores.iloc[i]['year']

    cotacoes_empresa = cotacoes[cotacoes['ticker']==ticker]
    cotacoes_empresa_ano = cotacoes_empresa[cotacoes_empresa['year']==ano]

    try:
        pl_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['lpa']
        pl_min = cotacoes_empresa_ano['min'].values[0]/indicadores.iloc[i]['lpa']
        pl_clo = cotacoes_empresa_ano['close'].values[0]/indicadores.iloc[i]['lpa']
    except:
        pl_max = 0
        pl_min = 0
        pl_clo = 0

    try:
        pvp_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['vpa']
        pvp_min = cotacoes_empresa_ano['min'].values[0]/indicadores.iloc[i]['vpa']
        pvp_clo = cotacoes_empresa_ano['close'].values[0]/indicadores.iloc[i]['vpa']
    except:
        pvp_max = 0
        pvp_min = 0
        pvp_clo = 0

    try:
        dy_max = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['max'].values[0]
        dy_min = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['min'].values[0]
        dy_clo = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['close'].values[0]
    except:
        dy_max = 0
        dy_min = 0
        dy_clo = 0

    dicionario = {
        'ticker': ticker,
        'year': ano,

        'pl':pl_clo,
        'pl_max': pl_max,
        'pl_min': pl_min,

        'pvp':pvp_clo,
        'pvp_max': pvp_max,
        'pvp_min': pvp_min,

        'dy':dy_clo,
        'dy_max': dy_max,
        'dy_min': dy_min
    }

    indicadores_values = pd.DataFrame(dicionario, index=[0])
    df_list_indicadores_val.append(indicadores_values)

C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\1100314948.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  pvp_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['vpa']
C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\1100314948.py:20: RuntimeWarning: divide by zero encountered in scalar divide
  pvp_min = cotacoes_empresa_ano['min'].values[0]/indicadores.iloc[i]['vpa']
C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\1100314948.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  pvp_clo = cotacoes_empresa_ano['close'].values[0]/indicadores.iloc[i]['vpa']
C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\1100314948.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  pl_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['lpa']
C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\1100314948.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  pl_min = cotacoes_empresa_ano['min'].values[0]/in

In [196]:
faixas_indicadores = pd.concat(df_list_indicadores_val)

In [197]:
faixas_indicadores

,ticker,year,pl,pl_max,pl_min,pvp,pvp_max,pvp_min,dy,dy_max,dy_min
0,CMIG4,2015,5.465527,14.336978,5.135634,1.039016,2.725510,0.976302,0.198960,0.075847,0.211741
0,CMIG4,2016,50.701614,65.563565,26.961948,1.312207,1.696848,0.697801,0.167805,0.129767,0.315556
0,CMIG4,2017,15.099318,26.462269,11.297015,1.055358,1.849564,0.789598,0.070905,0.040458,0.094770
0,CMIG4,2018,17.517810,17.530449,8.164867,1.914196,1.915577,0.892187,0.092973,0.092906,0.199475
0,CMIG4,2019,9.503302,10.847134,8.407562,1.885229,2.151813,1.667860,0.105165,0.092136,0.118871
...,...,...,...,...,...,...,...,...,...,...,...
0,TIMS3,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,TIMS3,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,TIMS3,2020,19.235763,19.879144,14.994704,1.529792,1.580959,1.192507,0.037527,0.036312,0.048141
0,TIMS3,2021,inf,inf,inf,inf,inf,inf,0.032915,0.029147,0.038819


In [198]:
df_list_faixa_media = []
for ticker in faixas_indicadores.ticker.unique():
    df = faixas_indicadores[faixas_indicadores['ticker']==ticker]
    dicionario = {'ticker': ticker}
    for coluna in df.columns:
        if coluna in ['ticker', 'year']:
            pass
        else:
            valor = df[coluna].mean()
            dicionario[coluna] = valor
    df = pd.DataFrame(dicionario,index=[0])
    df_list_faixa_media.append(df)

In [199]:
faixas_indicadores_medias = pd.concat(df_list_faixa_media)

In [200]:
faixas_indicadores_medias

,ticker,pl,pl_max,pl_min,pvp,pvp_max,pvp_min,dy,dy_max,dy_min
0,CMIG4,15.394268,20.581820,9.701225,1.456076,1.963351,1.029048,0.146837,0.106908,0.202970
0,CMIG3,16.692189,22.669076,10.577898,1.677136,2.247530,1.135949,0.117186,0.084170,0.166192
0,TAEE11,23.343293,27.196038,19.335831,5.323586,6.146740,4.347887,0.324237,0.276826,0.397000
0,TAEE3,6.030254,7.208846,5.044934,1.420915,1.661071,1.162918,0.229459,0.195530,0.281992
0,TAEE4,5.886004,7.498779,4.911851,1.392965,1.695827,1.135859,0.234989,0.196148,0.285788
0,CPLE6,57.704133,76.541059,38.397392,5.690345,7.898657,3.612224,0.159679,0.092339,0.212702
0,CPLE3,49.524451,65.059766,31.792086,5.055537,7.118394,3.144069,0.165736,0.096754,0.221493
0,CSMG3,-43.579471,-85.843324,-22.120452,2.014649,2.705679,1.320294,0.096493,0.042375,0.119983
0,SAPR11,58.212801,76.441193,42.524169,8.889429,11.508550,6.378085,0.113335,0.091790,0.145338
0,SAPR3,15.835525,24.132638,10.759661,2.284566,3.446578,1.549581,0.190940,0.131003,0.268449
